# Tests for Dense Layer

To make sure that the output from the GPU is correct, you should test them against what you get with the original CPU implementation. The following code is what you get from the original implementation on the CPU.

You should do the same for any other layer you implement.

In [1]:
import numpy as np

import pyopencl as cl
import pyopencl.array as cl_array
import PyNetwork
from PyNetwork.layers.Dense_GPU import Dense_GPU

In [2]:
input_shape = (10,)
output_nodes = 100

layer = PyNetwork.layers.Dense(output_nodes, 'relu')
layer.build(input_shape)

In [3]:
platform = cl.get_platforms()
devices = platform[0].get_devices()
context = cl.Context(devices)
queue = cl.CommandQueue(context)

layer_gpu = Dense_GPU(context = context, queue = queue, hidden_nodes = output_nodes, activation_function='relu')
layer_gpu.build(input_shape)

# Test Predict

In [4]:
n_datapoints = 20
z_in = np.random.rand(n_datapoints, *input_shape).astype(np.float32) - 0.5

a_true, z_out_true = layer.predict(z_in, output_only=False)

In [8]:
z_in_gpu = cl_array.to_device(queue, z_in)
a_true_gpu, z_out_true_gpu = layer_gpu.predict(z_in_gpu, output_only=False)

# Test Backprop

In [15]:
g_prime = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
new_delta = np.random.rand(n_datapoints, output_nodes).astype(np.float32)

delta_true = layer.get_delta_backprop_(g_prime, new_delta)

In [19]:
g_prime_gpu = cl_array.to_device(queue, g_prime)
new_delta_gpu =cl_array.to_device(queue, new_delta)

delta_true_gpu = layer_gpu.get_delta_backprop_(g_prime_gpu, new_delta_gpu)

# Test Weight Gradients

In [24]:
delta = np.random.rand(n_datapoints, *input_shape).astype(np.float32)
prev_z = np.random.rand(n_datapoints, output_nodes).astype(np.float32)

delta_grad_true, w_grad_true = layer.get_weight_grad_(delta, prev_z)

In [25]:
delta_gpu= cl_array.to_device(queue, delta)
prev_z_gpu =cl_array.to_device(queue, prev_z)

delta_grad_true_gpu, w_grad_true_gpu = layer_gpu.get_weight_grad_(delta_gpu, prev_z_gpu)